#### InceptionNet
    InceptionNet诞生于2014年，当年ImageNet竞赛冠军，Top5错误率为6.67%
    Szegedy C, Liu W, Jia Y, et al. Going Deeper with Convolutions. In CVPR, 2015.
    
    InceptionNet引入Inception结构块
    在同一层网络内使用不同尺寸的卷积核，提升了模型感知力
    使用了批标准化，缓解了梯度消失
    
    使用1*1卷积核，作用到输入特征图的每一个像素点，
    通过设定少于输入特征图深度的1*1卷积核个数，
    减少了输出特征图深度，起到了降维的作用。
    减少了参数量核计算量。
    
###### Inception结构块
    一个Inception结构块包含四个分支：
    ①1*1卷积核→卷积连接器
    ②1*1卷积核→3*3卷积核→卷积连接器
    ③1*1卷积核→5*5卷积核→卷积连接器
    ④3*3maxpooling→1*1卷积核→卷积连接器
    
    送到卷积连接器的四个特征数据尺寸相同，卷积连接器会把收到的四路特征数据按深度方向拼接，形成Inception结构块的输出
    
    结构：
    ①
    C（核：16*1*1，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ②
    C（核：16*1*1，步长：1，填充：same）         C（核：16*3*3，步长：1，填充：same ）
    B（Yes）                            B（Yes）
    A（relu）                           A（relu）
    P（None）                           P（None）
    D（None）                           D（None）
    
    ③
    C（核：16*1*1，步长：1，填充：same）         C（核：16*5*5，步长：1，填充：same ）
    B（Yes）                            B（Yes）
    A（relu）                           A（relu）
    P（None）                           P（None）
    D（None）                           D（None）
    
    ④
    C（None）                           C（核：16*1*1，步长：1，填充：same ）
    B（None）                           B（Yes）
    A（None）                           A（relu）
    P（Max，核：3*3，步长：1，填充：same）        P（None）
    D（None）                           D（None）
    
###### Inception结构
    in：32*32
    
    C（核：16*3*3，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    block_0：
        ①②③④
        ①②③④
    
    block_1：
        ①②③④
        ①②③④
        
    global avgpooling
    
    Dense 10

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense, \
    GlobalAveragePooling2D
from tensorflow.keras import Model

np.set_printoptions(threshold=np.inf)

cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

####定义一个卷积、批标准化、Relu激活函数操作的类，减少代码数量，增加代码的可读性####
class ConvBNRelu(Model):
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):#默认卷积核3*3，步长1，全零填充
        super(ConvBNRelu, self).__init__()
        self.model = tf.keras.models.Sequential([
            Conv2D(ch, kernelsz, strides=strides, padding=padding),
            BatchNormalization(),
            Activation('relu')
        ])

    def call(self, x):
        x = self.model(x, training=False) #在training=False时，BN通过整个训练集计算均值、方差去做批归一化，training=True时，通过当前batch的均值、方差去做批归一化。推理时 training=False效果好
        return x


class InceptionBlk(Model):
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        self.ch = ch
        self.strides = strides
        
        self.c1 = ConvBNRelu(ch, kernelsz=1, strides=strides)
        
        self.c2_1 = ConvBNRelu(ch, kernelsz=1, strides=strides)
        self.c2_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.c3_1 = ConvBNRelu(ch, kernelsz=1, strides=strides)
        self.c3_2 = ConvBNRelu(ch, kernelsz=5, strides=1)
        
        self.p4_1 = MaxPool2D(3, strides=1, padding='same')#最大池化
        self.c4_2 = ConvBNRelu(ch, kernelsz=1, strides=strides)

    def call(self, x):
        x1 = self.c1(x)
        x2_1 = self.c2_1(x)
        x2_2 = self.c2_2(x2_1)
        x3_1 = self.c3_1(x)
        x3_2 = self.c3_2(x3_1)
        x4_1 = self.p4_1(x)
        x4_2 = self.c4_2(x4_1)
        # concat along axis=channel
        x = tf.concat([x1, x2_2, x3_2, x4_2], axis=3)#将Inception结构块中4个分支的特征数据按照深度的方向拼接
        return x

####搭建InceptionNet网络结构####
class Inception10(Model):
    def __init__(self, num_blocks, num_classes, init_ch=16, **kwargs):#默认输出深度为16
        super(Inception10, self).__init__(**kwargs)
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_blocks = num_blocks
        self.init_ch = init_ch
        
        self.c1 = ConvBNRelu(init_ch)
        
        self.blocks = tf.keras.models.Sequential()
        for block_id in range(num_blocks):
            for layer_id in range(2):
                if layer_id == 0:
                    block = InceptionBlk(self.out_channels, strides=2)#步长为2，输出特征图尺寸减半
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                self.blocks.add(block)
            # enlarger out_channels per block
            #加深输出特征图深度，尽可能保持特征抽取中信息的承载量一致
            # block_0的通道数16，
            # block_1通道数32，block1经过四个分支后输出深度为4*32=128
            self.out_channels *= 2
        #将128个通道的数据平均池化    
        self.p1 = GlobalAveragePooling2D()
        #送入10个分类的全连接
        self.f1 = Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.blocks(x)
        x = self.p1(x)
        y = self.f1(x)
        return y


model = Inception10(num_blocks=2, num_classes=10)#实例化Inception10的类，指定InceptionNet的block数为2，指定网络为十分类

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = "./checkpoint/Inception10.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])
model.summary()

# print(model.trainable_variables)
file = open('./weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

###############################################    show   ###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
